In [22]:
pip install yfinance bs4

Note: you may need to restart the kernel to use updated packages.


# Import libraries
yfinance for extracting stock data

In [23]:
import pandas as pd

import yfinance as yf
import requests
from bs4 import BeautifulSoup

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [24]:
def plot_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price ($)", "Historical Revenue ($)"), vertical_spacing = .5)
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data.Date, infer_datetime_format=True), y=stock_data.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data.Date, infer_datetime_format=True), y=revenue_data.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($ Millions)", row=2, col=1)
    fig.update_layout(showlegend=False, height=1000, title=stock, xaxis_rangeslider_visible=True)
    fig.show()

# Tesla Stock Data

In [45]:
# Using the Ticker function to create a ticker object.
# ticker symbol of tesla is TSLA
tesla_data = yf.Ticker('TSLA')

# history function helps to extract stock information.
# setting period parameter to max to get information for the maximum amount of time.
tsla_data = tesla_data.history(period='max')

# Resetting the index
tsla_data.reset_index(inplace=True)

# display the last five rows
tsla_data.tail()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
3629,2024-11-27 00:00:00-05:00,341.799988,342.549988,326.589996,332.890015,57896400,0.0,0.0
3630,2024-11-29 00:00:00-05:00,336.079987,345.450012,334.649994,345.160004,37167600,0.0,0.0
3631,2024-12-02 00:00:00-05:00,352.380005,360.000000,351.149994,357.089996,77986500,0.0,0.0
3632,2024-12-03 00:00:00-05:00,351.799988,355.690002,348.200012,351.420013,58081600,0.0,0.0
3633,2024-12-04 00:00:00-05:00,353.040009,354.820007,348.600006,352.119995,28301012,0.0,0.0


In [26]:
# Webscraping to extract Tesla revenue data
# using requests library to download the webpage
url='https://companiesmarketcap.com/tesla/revenue/'

# Save the text of the response
html_text = requests.get(url).text

# Parse the html data using beautiful_soup.
soup = BeautifulSoup(html_text, 'html5lib')
soup

<!DOCTYPE html>
<html lang="en-US"><head>
<title>Tesla (TSLA) - Revenue</title>
<meta charset="utf-8"/>
<link href="https://companiesmarketcap.com/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="/img/icons/apple-icon-57x57.png" rel="apple-touch-icon" sizes="57x57"/>
<link href="/img/icons/apple-icon-60x60.png" rel="apple-touch-icon" sizes="60x60"/>
<link href="/img/icons/apple-icon-72x72.png" rel="apple-touch-icon" sizes="72x72"/>
<link href="/img/icons/apple-icon-76x76.png" rel="apple-touch-icon" sizes="76x76"/>
<link href="/img/icons/apple-icon-114x114.png" rel="apple-touch-icon" sizes="114x114"/>
<link href="/img/icons/apple-icon-120x120.png" rel="apple-touch-icon" sizes="120x120"/>
<link href="/img/icons/apple-icon-144x144.png" rel="apple-touch-icon" sizes="144x144"/>
<link href="/img/icons/apple-icon-152x152.png" rel="apple-touch-icon" sizes="152x152"/>
<link href="/img/icons/apple-icon-180x180.png" rel="apple-touch-icon" sizes="180x180"/>
<link href="/img/icons

In [31]:
# Using beautiful soup extract the table with Tesla Quarterly Revenue.
dates = []
revenues = []

tables = soup.find_all('table')
table_index=0

for index, table in enumerate(tables):
    if ('Tesla Quarterly Revenue'in str(table)):
        table_index=index
        
for row in tables[table_index].tbody.find_all("tr"):
    col = row.find_all("td")
    if (col!=[]):
        date =col[0].text.strip().strip('(TTM)')
        # to remove comma and dollar sign
        revenue =col[1].text.replace("$", "").replace(",", "").split(" ")[0]
        dates.append(date)
        revenues.append(revenue)

tsla_revenue = pd.DataFrame(columns=["Date", "Revenue"], data=list(zip(dates, revenues)))
# displaying dataframe
tsla_revenue

,Date,Revenue
0,2024,97.15
1,2023,96.77
2,2022,81.46
3,2021,53.82
4,2020,31.53
5,2019,24.57
6,2018,21.46
7,2017,11.75
8,2016,7.00
9,2015,4.04


In [32]:
plot_graph(tsla_data, tsla_revenue, 'Tesla Historical Share Price & Revenue')

/var/folders/nv/cddx4y_s46n_4yhxp7v2bb540000gn/T/ipykernel_25206/811683721.py:3: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

/Users/byron/opt/anaconda3/envs/ODLG4/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

/var/folders/nv/cddx4y_s46n_4yhxp7v2bb540000gn/T/ipykernel_25206/811683721.py:4: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html.

# GameStop stock

In [43]:
#  ticker symbol of GameStop is GME
gamestop = yf.Ticker('GME')

# extracting stock information
gme_data=gamestop.history(period='max')

#reset the index
gme_data.reset_index(inplace=True)
gme_data.tail()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
5737,2024-11-27 00:00:00-05:00,30.799999,31.549999,30.510,30.889999,12895100,0.0,0.0
5738,2024-11-29 00:00:00-05:00,31.010000,31.080000,28.850,29.049999,10473700,0.0,0.0
5739,2024-12-02 00:00:00-05:00,28.959999,29.370001,26.750,27.270000,13576600,0.0,0.0
5740,2024-12-03 00:00:00-05:00,26.549999,27.590000,26.490,27.440001,5981200,0.0,0.0
5741,2024-12-04 00:00:00-05:00,27.070000,27.830000,26.638,27.410000,3214450,0.0,0.0


In [34]:
# Webscraping to Extract GME revenue data
# using requests library to download the webpage
url = 'https://companiesmarketcap.com/gamestop/revenue/'

# Save the text of the response
html_data = requests.get(url).text

# parse the html data
soup = BeautifulSoup(html_data, 'html5lib')
soup

<!DOCTYPE html>
<html lang="en-US"><head>
<title>GameStop
 (GME) - Revenue</title>
<meta charset="utf-8"/>
<link href="https://companiesmarketcap.com/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="/img/icons/apple-icon-57x57.png" rel="apple-touch-icon" sizes="57x57"/>
<link href="/img/icons/apple-icon-60x60.png" rel="apple-touch-icon" sizes="60x60"/>
<link href="/img/icons/apple-icon-72x72.png" rel="apple-touch-icon" sizes="72x72"/>
<link href="/img/icons/apple-icon-76x76.png" rel="apple-touch-icon" sizes="76x76"/>
<link href="/img/icons/apple-icon-114x114.png" rel="apple-touch-icon" sizes="114x114"/>
<link href="/img/icons/apple-icon-120x120.png" rel="apple-touch-icon" sizes="120x120"/>
<link href="/img/icons/apple-icon-144x144.png" rel="apple-touch-icon" sizes="144x144"/>
<link href="/img/icons/apple-icon-152x152.png" rel="apple-touch-icon" sizes="152x152"/>
<link href="/img/icons/apple-icon-180x180.png" rel="apple-touch-icon" sizes="180x180"/>
<link href="/img/ic

In [37]:
# Using beautiful soup extract the table with GameStop Quarterly Revenue
dates, revenues = [], []

tables = soup.find_all('table')

table_index=0
for index, table in enumerate(tables):
    if ('GameStop Quarterly Revenue'in str(table)):
        table_index=index
        
for row in tables[table_index].tbody.find_all("tr"):
    col = row.find_all("td")
    if (col!=[]):
        date =col[0].text.strip().strip('(TTM)')
        # comma and dollar sign is removed
        revenue =col[1].text.replace("$", "").replace(",", "").strip(" B")
        dates.append(date)
        revenues.append(revenue)

gme_revenue = pd.DataFrame(columns=["Date", "Revenue"], data=list(zip(dates, revenues)))     
gme_revenue.head()

,Date,Revenue
0,2024,4.83
1,2023,5.27
2,2022,5.92
3,2021,6.01
4,2020,5.08


In [38]:
plot_graph(gme_data, gme_revenue, 'GameStop')

/var/folders/nv/cddx4y_s46n_4yhxp7v2bb540000gn/T/ipykernel_25206/811683721.py:3: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

/Users/byron/opt/anaconda3/envs/ODLG4/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

/var/folders/nv/cddx4y_s46n_4yhxp7v2bb540000gn/T/ipykernel_25206/811683721.py:4: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html.